# 04.2- Agents

Let's take a look at agents, and expand the capabilities of LLMs.

### AgentType

There are several types of agents, each optimized for what we need. An agent can store past chat information or can enforce each call to the LLM as an independent one.  
It is up to you to decide what you need in your future projects, but here we will focus on agents capable of maintaining a chat.

### Tools

The `tools` are a set of options one uses to give further capabilities to the agent: parsing wikipedia, using google for searches, etc..  
Several models are trained with a 1-2 year delay in data, like chatgpt-3.5. It was only trained with data up to 2022. Some tools may help to overcome this limitation with active searches. 

In [4]:
!pip install -qU numexpr
!pip install -qU langchainhub
#!pip install langchain_openai

In [5]:
from langchain.agents import AgentType # The type of agent
from langchain.memory import ConversationBufferMemory # Memory
from langchain.agents import initialize_agent 
from langchain.agents import load_tools #Additional tools
from langchain_openai import ChatOpenAI # Our LLM
from IPython.display import Markdown #Pretty display

In [6]:
# Let's create a memory instance for our agent
memory = ConversationBufferMemory(memory_key="chat_history")

In [7]:
llm=ChatOpenAI(temperature=0.1)
tools = load_tools(["llm-math"], llm=llm) 
agent_chain = initialize_agent(tools, 
                               llm,
                               agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, 
                               verbose=True, 
                               memory=memory)

C:\Users\Guimarais\AppData\Local\Temp\ipykernel_13088\3558800956.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_chain = initialize_agent(tools,


In [8]:
answer = agent_chain.invoke(input="What are some good Portuguese food recipes?", handle_parsing_errors=True)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Some popular Portuguese food recipes include Bacalhau à Brás (salted cod with potatoes and eggs), Caldo Verde (kale soup), and Pastéis de Nata (custard tarts). Would you like more information on any of these recipes or other Portuguese dishes?

> Finished chain.


In [9]:
Markdown(answer['output'])

Some popular Portuguese food recipes include Bacalhau à Brás (salted cod with potatoes and eggs), Caldo Verde (kale soup), and Pastéis de Nata (custard tarts). Would you like more information on any of these recipes or other Portuguese dishes?

In [10]:
answer = agent_chain.run("Which one of those dishes has the least fat?")

C:\Users\Guimarais\AppData\Local\Temp\ipykernel_13088\845632627.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = agent_chain.run("Which one of those dishes has the least fat?")




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Fat content of Bacalhau à Brás, Caldo Verde, and Pastéis de Nata
Observation: Answer: 35
Thought:Do I need to use a tool? No
AI: The dish with the least fat content among Bacalhau à Brás, Caldo Verde, and Pastéis de Nata is Caldo Verde, which is a kale soup.

> Finished chain.


Now the history of our conversation is stored and you can build upon what you have chatted.

In [11]:
agent_chain.run("Give me a grocery shopping list to make that dish")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Grocery shopping list for Caldo Verde recipe

ValueError: unknown format from LLM: I'm sorry, but the question you provided does not seem to be a math problem that can be translated into a single line mathematical expression for execution using Python's numexpr library. Could you please provide a different math problem for me to solve?

---
### Other applications: Analyzing documents

The Langchain framework has a lot of code already specialized in tasks meant to be done over and over.  
In this example we go over a text file that we have seen before, the play from William Shakespeare.

In [12]:
import requests

# Let's get our Shakespeare back
response = requests.get("https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")

# There is a size limit for inputs
shakespeare = response.text[0:10000]

In [13]:
from langchain.chains import AnalyzeDocumentChain

In [14]:
from langchain.chains.question_answering import load_qa_chain

qa_chain = load_qa_chain(llm)
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

C:\Users\Guimarais\AppData\Local\Temp\ipykernel_13088\1109563361.py:3: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  qa_chain = load_qa_chain(llm)
C:\Users\Guimarais\AppData\Local\Temp\ipykernel_13088\1109563361.py:4: LangChainDeprecationWarning: The class `AnalyzeDocumentChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~example in API reference with more detail: https://api.python.langchain.com/en/latest/chains/langch

In [15]:
qa_document_chain.run(
    input_document=shakespeare,
    question="What is this work?",
)

'The text provided is an excerpt from the play "Coriolanus" by William Shakespeare.'

In [16]:
qa_document_chain.run(
    input_document=shakespeare,
    question="Describe the main protagonist of this work.",
)

'The main protagonist of this work is Caius Marcius, also known as Coriolanus. He is depicted as a skilled and proud Roman general who is initially revered for his military prowess but later faces challenges due to his disdain for the common people and his inability to navigate the political landscape effectively. Coriolanus is a complex character who struggles with his own pride and loyalty to Rome, ultimately leading to his downfall.'

---
### PDF documents

In [ ]:
#!pip install pypdf
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
# Windows
#!cURL https://media.wizards.com/images/magic/resources/rules/EN_MTGM14_PrintedRulebook_LR.pdf >> EN_MTGM14_PrintedRulebook_LR.pdf

In [ ]:
# MAC/Linux
#!wget https://media.wizards.com/images/magic/resources/rules/EN_MTGM14_PrintedRulebook_LR.pdf

In [ ]:
loader = PyPDFLoader("EN_MTGM14_PrintedRulebook_LR.pdf")
pages = loader.load_and_split()

In [ ]:
pages[10]

A PDF is usually text+images. Ideally, you send both the images and the the text to the model for analysis.  
But you need a multi-modal model for that, which we will not cover due to time constraints.  
Here, we are just going to get the text and run some simple queries.

In [ ]:
pages[10].to_json()['kwargs']['page_content']

In [ ]:
text = []

# Append all pages on a list and then join all list elements
for page in pages:
    text.append(page.to_json()['kwargs']['page_content'])

all_text = '\n'.join(text)

In [ ]:
# Number of pages, number of characters
len(text), len(all_text)

The document is too big for our model, we need to focus on some pages only.

In [ ]:
text = []

# Append all pages on a list and then join all list elements
for pagenumber in range(2,6):
    text.append(pages[pagenumber].to_json()['kwargs']['page_content'])

all_text = '\n'.join(text)

In [ ]:
# Number of pages, number of characters
len(text), len(all_text)

In [ ]:
text

In [ ]:
answer = qa_document_chain.run(input_document=all_text,
                               question="What are the parts of a card?",)

Markdown(answer)

To analyse more content, one must use models capable of handling more tokens.